In [ ]:
import os
import shutil
import random
from google.colab import drive

def create_dataset_structure():
    # 데이터셋 디렉토리 구조 생성
    dirs = [
        '/content/drive/MyDrive/safehouse/dataset/images/train',
        '/content/drive/MyDrive/safehouse/dataset/images/val',
        '/content/drive/MyDrive/safehouse/dataset/labels/train',
        '/content/drive/MyDrive/safehouse/dataset/labels/val'
    ]
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)

def split_and_move_files(image_dir, label_dir, train_ratio=0.8):
    # 이미지와 라벨 파일 리스트 가져오기
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.tiff'))]
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 파일 쌍 만들기
    pairs = []
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        txt_file = base_name + '.txt'
        if txt_file in label_files:
            pairs.append((img_file, txt_file))

    # 랜덤 섞기
    random.shuffle(pairs)

    # train/val 분할
    split_idx = int(len(pairs) * train_ratio)
    train_pairs = pairs[:split_idx]
    val_pairs = pairs[split_idx:]

    # 파일 이동
    for pair in train_pairs:
        img_file, txt_file = pair
        # 이미지 이동
        shutil.copy2(
            os.path.join(image_dir, img_file),
            os.path.join('/content/drive/MyDrive/safehouse/dataset/images/train', img_file)
        )
        # 라벨 이동
        shutil.copy2(
            os.path.join(label_dir, txt_file),
            os.path.join('/content/drive/MyDrive/safehouse/dataset/labels/train', txt_file)
        )

    for pair in val_pairs:
        img_file, txt_file = pair
        # 이미지 이동
        shutil.copy2(
            os.path.join(image_dir, img_file),
            os.path.join('/content/drive/MyDrive/safehouse/dataset/images/val', img_file)
        )
        # 라벨 이동
        shutil.copy2(
            os.path.join(label_dir, txt_file),
            os.path.join('/content/drive/MyDrive/safehouse/dataset/labels/val', txt_file)
        )

    return len(pairs), len(train_pairs), len(val_pairs)

def process_all_folders():
    # 기본 경로
    base_image_path = "/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data"
    base_label_path = "/content/drive/MyDrive/safehouse/concrete_txt"

    # 처리할 폴더 목록
    folders = [
        'TL_지상시설물_도로교량_교각',
        'TL_지상시설물_철도교량_교각',
        'TL_지상시설물_철도교량_상부구조',
        'TL_지상시설물_도로교량_상부구조',
        'TL_지상시설물_옹벽'
    ]

    # 데이터셋 구조 생성
    create_dataset_structure()

    # 각 폴더별 처리
    total_stats = {'total': 0, 'train': 0, 'val': 0}

    for folder in folders:
        try:
            image_dir = os.path.join(base_image_path, folder)
            label_dir = os.path.join(base_label_path, folder)

            if os.path.exists(image_dir) and os.path.exists(label_dir):
                print(f"\n처리 중인 폴더: {folder}")
                total, train, val = split_and_move_files(image_dir, label_dir)
                print(f"처리된 파일: 전체 {total}, 학습 {train}, 검증 {val}")

                total_stats['total'] += total
                total_stats['train'] += train
                total_stats['val'] += val
            else:
                print(f"폴더를 찾을 수 없음: {folder}")

        except Exception as e:
            print(f"폴더 처리 중 오류 발생 {folder}: {str(e)}")

    print("\n=== 전체 처리 결과 ===")
    print(f"총 파일 수: {total_stats['total']}")
    print(f"학습 데이터: {total_stats['train']}")
    print(f"검증 데이터: {total_stats['val']}")

# 실행
try:
    process_all_folders()
except Exception as e:
    print(f"전체 처리 중 오류 발생: {str(e)}")



처리 중인 폴더: TL_지상시설물_도로교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_상부구조
처리된 파일: 전체 255, 학습 204, 검증 51

처리 중인 폴더: TL_지상시설물_도로교량_상부구조
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_옹벽
처리된 파일: 전체 1000, 학습 800, 검증 200

=== 전체 처리 결과 ===
총 파일 수: 4255
학습 데이터: 3404
검증 데이터: 851


In [ ]:
import os
import zipfile

def unzip_file(zip_path, extract_to):
    """압축 파일을 지정된 폴더에 풀기"""
    try:
        # 압축 해제할 폴더가 없으면 생성
        if not os.path.exists(extract_to):
            os.makedirs(extract_to)
            print(f"폴더 생성됨: {extract_to}")

        # 압축 파일 열기 및 해제
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"압축 해제 완료: {zip_path} -> {extract_to}")

    except FileNotFoundError:
        print(f"압축 파일을 찾을 수 없습니다: {zip_path}")
    except Exception as e:
        print(f"에러 발생: {str(e)}")

# 사용 예시
zip_path = "/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/TL_지상시설물_도로교량_교각.zip"  # 실제 zip 파일 경로로 변경
extract_to = "/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/TL_지상시설물_도로교량_교각"  # 압축을 해제할 폴더 경로

unzip_file(zip_path, extract_to)


폴더 생성됨: /content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/TL_지상시설물_도로교량_교각
압축 해제 완료: /content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/TL_지상시설물_도로교량_교각.zip -> /content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/TL_지상시설물_도로교량_교각


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.9/906.9 kB 21.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# 모델 생성
model = YOLO('yolov8n.pt')  # 또는 yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt

# 학습 실행
results = model.train(
    data='/content/drive/MyDrive/safehouse/dataset/data.yaml',
    epochs=20,
    imgsz=640,
    batch=16,
    patience=50,
    save=True,
    device='0'  # GPU 사용
)


Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/safehouse/dataset/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 17.2MB/s]


AMP: checks passed ✅


...

                   all        897       3014      0.104     0.0692     0.0339     0.0137

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      2.18G      2.354      3.091      1.868         34        640: 100%|██████████| 216/216 [01:50<00:00,  1.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:17<00:00,  1.66it/s]


                   all        897       3014      0.098     0.0753     0.0352     0.0146

20 epochs completed in 0.729 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 6.2MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:17<00:00,  1.62it/s]


                   all        897       3014     0.0979     0.0753     0.0352     0.0146
                level1        558       1747      0.149     0.0927     0.0472     0.0195
                level2        426       1163      0.145      0.133     0.0555     0.0228
                level3         53        104          0          0    0.00286    0.00138
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train7


In [ ]:
model = YOLO('yolov8s.pt')  # 또는 yolov8m.pt
results = model.train(
    data='/content/drive/MyDrive/safehouse/dataset/data.yaml',
    epochs=50,  # 20 -> 50으로 증가
    imgsz=800,   # 640 -> 800으로 증가
    batch=16,    # 16 -> 32로 증가
    patience=50,
    save=True,
    device='0',
    lr0=0.001,   # 초기 학습률 추가
    lrf=0.01,    # 최종 학습률 추가
    warmup_epochs=5.0,  # 웜업 에폭 증가
    hsv_h=0.015,  # 색조 증강
    hsv_s=0.7,    # 채도 증강
    hsv_v=0.4,    # 명도 증강
    degrees=10.0,  # 회전 증강 추가
    translate=0.2, # 이동 증강 증가
    scale=0.5,    # 스케일 증강
    fliplr=0.5,   # 좌우 반전
    mosaic=1.0    # 모자이크 증강
)


Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/safehouse/dataset/data.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning /content/drive/MyDrive/safehouse/dataset/labels/train.cache... 3450 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3450/3450 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/safehouse/dataset/labels/val.cache... 897 images, 0 backgrounds, 0 corrupt: 100%|██████████| 897/897 [00:00<?, ?it/s]


Plotting labels to runs/detect/train9/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 800 train, 800 val
Using 2 dataloader workers
Logging results to runs/detect/train9
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        897       3014      0.132       0.15     0.0664     0.0283

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      5.88G      1.931      2.595      1.659         51        800: 100%|██████████| 216/216 [02:31<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:20<00:00,  1.44it/s]


                   all        897       3014      0.133      0.154     0.0677     0.0292

50 epochs completed in 2.505 hours.
Optimizer stripped from runs/detect/train9/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train9/weights/best.pt, 22.5MB

Validating runs/detect/train9/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:21<00:00,  1.35it/s]


                   all        897       3014      0.134      0.154     0.0679     0.0293
                level1        558       1747      0.184      0.197     0.0933     0.0389
                level2        426       1163      0.192      0.246        0.1     0.0434
                level3         53        104     0.0254     0.0192     0.0104    0.00551
Speed: 0.3ms preprocess, 3.8ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/train9


# 역시 너무 쓰레기라 바꿀게요 다시.. 세그멘테이션

In [ ]:
import os
import json
import numpy as np
from shapely.geometry import Polygon

def convert_points_to_polygon(points):
    """포인트 리스트를 Polygon 객체로 변환"""
    return Polygon(points)

def normalize_coordinates(polygon, img_width, img_height):
    """다각형 좌표를 정규화"""
    normalized_coords = []
    for x, y in polygon.exterior.coords[:-1]:  # 마지막 점은 첫 점과 같으므로 제외
        normalized_coords.extend([x / img_width, y / img_height])
    return normalized_coords

def get_combined_class_id(width_str):
    """균열 폭에 따른 통합 클래스 ID 반환"""
    if '0.1mm 이하' in width_str:
        return 0
    elif '0.1mm 초과, 0.3mm 이하' in width_str:
        return 1
    elif '0.3mm 초과' in width_str:
        return 2
    return 0

def convert_points_to_polygon(points):
    if len(points) < 4:
        return None  # 4개 미만의 좌표인 경우 None 반환
    return Polygon(points)

def convert_json_to_yolo_segmentation(json_path, output_dir):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        img_width = data['image']['width']
        img_height = data['image']['height']

        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(json_path))[0]
        txt_path = os.path.join(output_dir, f"{base_name}.txt")

        with open(txt_path, 'w') as f:
            for annotation in data['image']['annotations']:
                if 'points' in annotation:
                    polygon = convert_points_to_polygon(annotation['points'])
                    if polygon is None:
                        continue  # 좌표가 부족한 경우 건너뛰기

                    normalized_coords = normalize_coordinates(polygon, img_width, img_height)
                    class_id = get_combined_class_id(annotation.get('width', ''))

                    line = f"{class_id} " + " ".join([f"{x:.6f}" for x in normalized_coords])
                    f.write(line + '\n')

    except Exception as e:
        print(f"Error processing {json_path}: {str(e)}")


def process_all_folders(input_folders, base_output_dir):
    """여러 입력 폴더의 JSON 파일들을 처리"""
    for folder_path in input_folders:
        folder_name = os.path.basename(folder_path)
        output_dir = os.path.join(base_output_dir, folder_name)
        os.makedirs(output_dir, exist_ok=True)

        if not os.path.exists(folder_path):
            print(f"입력 디렉토리가 존재하지 않습니다: {folder_path}")
            continue

        print(f"처리 중인 폴더: {folder_name}")
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                json_path = os.path.join(folder_path, filename)
                convert_json_to_yolo_segmentation(json_path, output_dir)

# 입력 폴더와 출력 폴더 설정
input_folders = [
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_도로교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_도로교량_상부구조',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_옹벽',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_철도교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_철도교량_상부구조'
]

output_base_dir = '/content/drive/MyDrive/safehouse/segmentation_labels'

# 모든 폴더 처리
process_all_folders(input_folders, output_base_dir)


처리 중인 폴더: TL_지상시설물_도로교량_교각
처리 중인 폴더: TL_지상시설물_도로교량_상부구조
처리 중인 폴더: TL_지상시설물_옹벽
처리 중인 폴더: TL_지상시설물_철도교량_교각
처리 중인 폴더: TL_지상시설물_철도교량_상부구조


In [ ]:
import os
import shutil
import random
from google.colab import drive

def create_dataset_structure():
    """데이터셋 디렉토리 구조 생성 및 초기화"""
    dirs = [
        '/content/drive/MyDrive/safehouse/dataset/images/train',
        '/content/drive/MyDrive/safehouse/dataset/images/val',
        '/content/drive/MyDrive/safehouse/dataset/labels/train',
        '/content/drive/MyDrive/safehouse/dataset/labels/val'
    ]
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
        # 기존 파일 삭제
        if os.path.exists(dir_path):
            for file in os.listdir(dir_path):
                file_path = os.path.join(dir_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)

def split_and_move_files(image_dir, label_dir, train_ratio=0.8):
    """이미지와 라벨 파일을 train/val로 분할하여 이동"""
    # 파일 리스트 생성
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.tiff'))]
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 매칭되는 파일 쌍 찾기
    pairs = []
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        txt_file = base_name + '.txt'
        if txt_file in label_files:
            pairs.append((img_file, txt_file))

    # 랜덤 분할
    random.shuffle(pairs)
    split_idx = int(len(pairs) * train_ratio)
    train_pairs = pairs[:split_idx]
    val_pairs = pairs[split_idx:]

    # 파일 복사 함수
    def copy_pairs(pairs, img_dest, label_dest):
        for img_file, txt_file in pairs:
            shutil.copy2(
                os.path.join(image_dir, img_file),
                os.path.join(img_dest, img_file)
            )
            shutil.copy2(
                os.path.join(label_dir, txt_file),
                os.path.join(label_dest, txt_file)
            )

    # train/val 데이터 복사
    copy_pairs(train_pairs,
              '/content/drive/MyDrive/safehouse/dataset/images/train',
              '/content/drive/MyDrive/safehouse/dataset/labels/train')

    copy_pairs(val_pairs,
              '/content/drive/MyDrive/safehouse/dataset/images/val',
              '/content/drive/MyDrive/safehouse/dataset/labels/val')

    return len(pairs), len(train_pairs), len(val_pairs)

def main():
    # 데이터셋 구조 생성
    create_dataset_structure()

    # 처리할 폴더 목록
    folders = [
        'TL_지상시설물_도로교량_교각',
        'TL_지상시설물_철도교량_교각',
        'TL_지상시설물_철도교량_상부구조',
        'TL_지상시설물_도로교량_상부구조',
        'TL_지상시설물_옹벽'
    ]

    # 기본 경로 설정
    base_image_path = "/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data"
    base_label_path = "/content/drive/MyDrive/safehouse/segmentation_labels"

    # 통계 정보 초기화
    total_stats = {'total': 0, 'train': 0, 'val': 0}

    # 각 폴더 처리
    for folder in folders:
        try:
            image_dir = os.path.join(base_image_path, folder)
            label_dir = os.path.join(base_label_path, folder)

            if os.path.exists(image_dir) and os.path.exists(label_dir):
                print(f"\n처리 중인 폴더: {folder}")
                total, train, val = split_and_move_files(image_dir, label_dir)
                print(f"처리된 파일: 전체 {total}, 학습 {train}, 검증 {val}")

                total_stats['total'] += total
                total_stats['train'] += train
                total_stats['val'] += val
            else:
                print(f"폴더를 찾을 수 없음: {folder}")

        except Exception as e:
            print(f"폴더 처리 중 오류 발생 {folder}: {str(e)}")

    # 최종 결과 출력
    print("\n=== 전체 처리 결과 ===")
    print(f"총 파일 수: {total_stats['total']}")
    print(f"학습 데이터: {total_stats['train']}")
    print(f"검증 데이터: {total_stats['val']}")

if __name__ == "__main__":
    main()



처리 중인 폴더: TL_지상시설물_도로교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_상부구조
처리된 파일: 전체 255, 학습 204, 검증 51

처리 중인 폴더: TL_지상시설물_도로교량_상부구조
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_옹벽
처리된 파일: 전체 1000, 학습 800, 검증 200

=== 전체 처리 결과 ===
총 파일 수: 4255
학습 데이터: 3404
검증 데이터: 851


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

# 데이터셋 설정 파일 생성
!echo "path: /content/drive/MyDrive/safehouse/dataset\ntrain: images/train\nval: images/val\nnc: 3\nnames: ['crack_1', 'crack_2', 'crack_3']" > /content/drive/MyDrive/safehouse/dataset/data.yaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.9/906.9 kB 22.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo task=segment mode=train model=yolov8s-seg.pt data=/content/drive/MyDrive/safehouse/dataset/data.yaml epochs=50 imgsz=640 batch=16 lr0=0.01 patience=50 workers=8 device=0 optimizer=Adam augment=True mosaic=1.0 save=True save_period=10 project=my_segmentation name=experiment1


100% 22.8M/22.8M [00:00<00:00, 251MB/s]
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/drive/MyDrive/safehouse/dataset/data.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=my_segmentation, name=experiment1, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

In [ ]:
# 학습된 모델 평가
!yolo task=segment mode=val model=/content/drive/MyDrive/safehouse/runs/my_segmentation/experiment1/weights/best.pt data=/content/drive/MyDrive/safehouse/dataset/data.yaml


/bin/bash: line 1: yolo: command not found


In [ ]:
pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.9/906.9 kB 23.3 MB/s eta 0:00:00


In [ ]:
!yolo task=segment mode=train model=yolov8m-seg.pt data=/content/drive/MyDrive/safehouse/dataset/data.yaml epochs=50 imgsz=640 batch=16 lr0=0.001 patience=40 workers=8 device=0 optimizer=Adam augment=True mosaic=1.0 mixup=0.5 copy_paste=0.3 save=True save_period=10 project=/content/drive/MyDrive/safehouse/my_segmentation name=experiment2


Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/drive/MyDrive/safehouse/dataset/data.yaml, epochs=50, time=None, patience=40, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=/content/drive/MyDrive/safehouse/my_segmentation, name=experiment25, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

In [ ]:
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO('/content/drive/MyDrive/safehouse/my_segmentation/experiment25/weights/best.pt')

# 검증 데이터셋에 대한 평가 수행
results = model.val(data='/content/drive/MyDrive/safehouse/dataset/data.yaml')

# 결과 출력
print(results)


Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,224,121 parameters, 0 gradients, 110.0 GFLOPs


val: Scanning /content/drive/MyDrive/safehouse/dataset/labels/val.cache... 851 images, 1 backgrounds, 0 corrupt: 100%|██████████| 851/851 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 54/54 [00:29<00:00,  1.84it/s]


                   all        851       2960      0.164       0.14     0.0704      0.032      0.117     0.0928     0.0376     0.0103
               crack_1        528       1675      0.198      0.175     0.0865     0.0398      0.142      0.118     0.0479     0.0136
               crack_2        397       1149        0.2      0.223      0.105     0.0492      0.142      0.146     0.0538      0.015
               crack_3         57        136     0.0924     0.0221     0.0193    0.00702     0.0683     0.0147     0.0112    0.00232
Speed: 0.5ms preprocess, 13.7ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/segment/val
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7804f7a96170>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall

# 억번째 다시

In [ ]:
import os
import shutil
import random
from google.colab import drive
import json
def process_crack_annotation(annotation):
    points = annotation.get('points', [])
    width_str = annotation.get('width', '')
    shape_type = annotation.get('shape', '')

    # 포인트가 2개 이상인 경우만 처리
    if len(points) < 2:
        return None

    # 폴리라인/폴리곤 좌표 정규화
    normalized_points = []
    for x, y in points:
        normalized_points.extend([
            float(x) / img_width,
            float(y) / img_height
        ])

    # 균열 폭에 따른 클래스 ID 할당
    class_id = 0
    if '0.1mm 초과, 0.3mm 이하' in width_str:
        class_id = 1
    elif '0.3mm 초과' in width_str:
        class_id = 2

    return class_id, normalized_points

def convert_json_to_yolo_segmentation(json_path, output_dir):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # image 키 안에 annotations가 있는 구조
        if 'image' not in data or 'annotations' not in data['image']:
            print(f"No annotations found in {json_path}")
            return

        img_width = data['image']['width']
        img_height = data['image']['height']

        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(json_path))[0]
        txt_path = os.path.join(output_dir, f"{base_name}.txt")

        with open(txt_path, 'w') as f:
            for annotation in data['image']['annotations']:
                if 'points' in annotation:
                    points = annotation['points']
                    if len(points) < 2:
                        continue

                    # 좌표 정규화
                    normalized_points = []
                    for point in points:
                        x, y = point
                        normalized_points.extend([
                            float(x) / img_width,
                            float(y) / img_height
                        ])

                    # 균열 폭에 따른 클래스 ID 결정
                    width_str = annotation.get('width', '')
                    class_id = 0  # 기본값
                    if '0.1mm 초과, 0.3mm 이하' in width_str:
                        class_id = 1
                    elif '0.3mm 초과' in width_str:
                        class_id = 2

                    # YOLO 형식으로 저장
                    line = f"{class_id} " + " ".join([f"{x:.6f}" for x in normalized_points])
                    f.write(line + '\n')

    except Exception as e:
        print(f"Error processing {json_path}: {str(e)}")



def process_all_folders(input_folders, base_output_dir):
    """여러 입력 폴더의 JSON 파일들을 처리"""
    for folder_path in input_folders:
        folder_name = os.path.basename(folder_path)
        output_dir = os.path.join(base_output_dir, folder_name)
        os.makedirs(output_dir, exist_ok=True)

        if not os.path.exists(folder_path):
            print(f"입력 디렉토리가 존재하지 않습니다: {folder_path}")
            continue

        print(f"처리 중인 폴더: {folder_name}")
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                json_path = os.path.join(folder_path, filename)
                convert_json_to_yolo_segmentation(json_path, output_dir)

# 입력 폴더와 출력 폴더 설정
input_folders = [
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_도로교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_도로교량_상부구조',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_옹벽',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_철도교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/라벨링데이터/TL_지상시설물_철도교량_상부구조'
]

output_base_dir = '/content/drive/MyDrive/safehouse/segmentation_label2'

# 모든 폴더 처리
process_all_folders(input_folders, output_base_dir)


처리 중인 폴더: TL_지상시설물_도로교량_교각
처리 중인 폴더: TL_지상시설물_도로교량_상부구조
처리 중인 폴더: TL_지상시설물_옹벽
처리 중인 폴더: TL_지상시설물_철도교량_교각
처리 중인 폴더: TL_지상시설물_철도교량_상부구조


In [ ]:

import os
from PIL import Image
import glob

def get_non_standard_image_sizes(folder_path):
    non_standard_sizes = []
    image_files = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_files.extend(glob.glob(os.path.join(folder_path, ext)))

    for img_path in image_files:
        try:
            with Image.open(img_path) as img:
                width, height = img.size
                if (width, height) != (1920, 1080):
                    print(f"비표준 이미지 발견: {os.path.basename(img_path)}, 크기: {width}x{height}")
                    non_standard_sizes.append((os.path.basename(img_path), width, height))
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
    return non_standard_sizes

# 폴더 검사
for folder in image_folders:
    print(f"\n폴더 검사 중: {os.path.basename(folder)}")
    non_standard = get_non_standard_image_sizes(folder)
    if not non_standard:
        print("모든 이미지가 1920x1080 규격입니다.")


# 입력 폴더 설정 (실제 이미지가 있는 폴더로 수정 필요)
image_folders = [
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data/TL_지상시설물_도로교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data/TL_지상시설물_도로교량_상부구조',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data/TL_지상시설물_옹벽',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data/TL_지상시설물_철도교량_교각',
    '/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data/TL_지상시설물_철도교량_상부구조'
]

for folder in image_folders:
    print(f"\n폴더 검사 중: {os.path.basename(folder)}")
    sizes = get_non_standard_image_sizes(folder)



폴더 검사 중: TL_지상시설물_도로교량_교각
모든 이미지가 1920x1080 규격입니다.

폴더 검사 중: TL_지상시설물_도로교량_상부구조
모든 이미지가 1920x1080 규격입니다.

폴더 검사 중: TL_지상시설물_옹벽
모든 이미지가 1920x1080 규격입니다.

폴더 검사 중: TL_지상시설물_철도교량_교각
모든 이미지가 1920x1080 규격입니다.

폴더 검사 중: TL_지상시설물_철도교량_상부구조
모든 이미지가 1920x1080 규격입니다.

폴더 검사 중: TL_지상시설물_도로교량_교각

폴더 검사 중: TL_지상시설물_도로교량_상부구조

폴더 검사 중: TL_지상시설물_옹벽

폴더 검사 중: TL_지상시설물_철도교량_교각

폴더 검사 중: TL_지상시설물_철도교량_상부구조


In [ ]:
import os
import shutil
import random
from google.colab import drive

def create_dataset_structure():
    """데이터셋 디렉토리 구조 생성 및 초기화"""
    dirs = [
        '/content/drive/MyDrive/safehouse/dataset/images/train',
        '/content/drive/MyDrive/safehouse/dataset/images/val',
        '/content/drive/MyDrive/safehouse/dataset/labels/train',
        '/content/drive/MyDrive/safehouse/dataset/labels/val'
    ]
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
        # 기존 파일 삭제
        if os.path.exists(dir_path):
            for file in os.listdir(dir_path):
                file_path = os.path.join(dir_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)

def split_and_move_files(image_dir, label_dir, train_ratio=0.8):
    """이미지와 라벨 파일을 train/val로 분할하여 이동"""
    # 파일 리스트 생성
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.tiff'))]
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 매칭되는 파일 쌍 찾기
    pairs = []
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        txt_file = base_name + '.txt'
        if txt_file in label_files:
            pairs.append((img_file, txt_file))

    # 랜덤 분할
    random.shuffle(pairs)
    split_idx = int(len(pairs) * train_ratio)
    train_pairs = pairs[:split_idx]
    val_pairs = pairs[split_idx:]

    # 파일 복사 함수
    def copy_pairs(pairs, img_dest, label_dest):
        for img_file, txt_file in pairs:
            shutil.copy2(
                os.path.join(image_dir, img_file),
                os.path.join(img_dest, img_file)
            )
            shutil.copy2(
                os.path.join(label_dir, txt_file),
                os.path.join(label_dest, txt_file)
            )

    # train/val 데이터 복사
    copy_pairs(train_pairs,
              '/content/drive/MyDrive/safehouse/dataset/images/train',
              '/content/drive/MyDrive/safehouse/dataset/labels/train')

    copy_pairs(val_pairs,
              '/content/drive/MyDrive/safehouse/dataset/images/val',
              '/content/drive/MyDrive/safehouse/dataset/labels/val')

    return len(pairs), len(train_pairs), len(val_pairs)

def main():
    # 데이터셋 구조 생성
    create_dataset_structure()

    # 처리할 폴더 목록
    folders = [
        'TL_지상시설물_도로교량_교각',
        'TL_지상시설물_철도교량_교각',
        'TL_지상시설물_철도교량_상부구조',
        'TL_지상시설물_도로교량_상부구조',
        'TL_지상시설물_옹벽'
    ]

    # 기본 경로 설정
    base_image_path = "/content/drive/MyDrive/Homeplus/data/Concrete/ConcreteCracks/원천데이터/soc/image_data"
    base_label_path = "/content/drive/MyDrive/safehouse/segmentation_label2"

    # 통계 정보 초기화
    total_stats = {'total': 0, 'train': 0, 'val': 0}

    # 각 폴더 처리
    for folder in folders:
        try:
            image_dir = os.path.join(base_image_path, folder)
            label_dir = os.path.join(base_label_path, folder)

            if os.path.exists(image_dir) and os.path.exists(label_dir):
                print(f"\n처리 중인 폴더: {folder}")
                total, train, val = split_and_move_files(image_dir, label_dir)
                print(f"처리된 파일: 전체 {total}, 학습 {train}, 검증 {val}")

                total_stats['total'] += total
                total_stats['train'] += train
                total_stats['val'] += val
            else:
                print(f"폴더를 찾을 수 없음: {folder}")

        except Exception as e:
            print(f"폴더 처리 중 오류 발생 {folder}: {str(e)}")

    # 최종 결과 출력
    print("\n=== 전체 처리 결과 ===")
    print(f"총 파일 수: {total_stats['total']}")
    print(f"학습 데이터: {total_stats['train']}")
    print(f"검증 데이터: {total_stats['val']}")

if __name__ == "__main__":
    main()



처리 중인 폴더: TL_지상시설물_도로교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_교각
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_철도교량_상부구조
처리된 파일: 전체 255, 학습 204, 검증 51

처리 중인 폴더: TL_지상시설물_도로교량_상부구조
처리된 파일: 전체 1000, 학습 800, 검증 200

처리 중인 폴더: TL_지상시설물_옹벽
처리된 파일: 전체 1000, 학습 800, 검증 200

=== 전체 처리 결과 ===
총 파일 수: 4255
학습 데이터: 3404
검증 데이터: 851
